#    DESAFÍO 1 
##   LIMPIEZA DE DATOS
###  Grupo 4: Juan Mutis, Joaquín Fernández, Rodrigo Arias, Ignacio Nasso 

<a id="section_toc"></a> 
## Indice

<a href="#section_intro">PARTE INICIAL. IMPORTACIÓN DE LIBRERÍAS, SAMPLEO DE DATOS, ANALISIS EXPLORATORIO</a>

<a href="#section_ubicacion">AJUSTE DE COLUMNAS REFERIDAS A LA UBICACIÓN.</a>

<a href="#section_descripcion">CAMPO DESCRIPTION - LIMPIEZA DE NULOS, CASEFOLD, UNICODE</a>

<a href="#section_rooms">LIMPIEZA DEL CAMPO 'rooms', IMPUTACIÓN DE DATOS FALTANTES.</a>

<a href="#section_surface">LIMPIEZA DE CAMPO 'surface_total_in_m2' y 'surface_covered_in_m2'</a>

<a href="#section_price">LIMPIEZA DE CAMPO 'price', 'price_aprox_usd' y 'currency'</a>

<a href="#section_drop">DROP DE OUTLIERS Y NULOS</a>

<a href="#section_graficos">GRÁFICOS POST DROPEO FINAL</a>

---

<a id="section_intro"></a>
### PARTE INICIAL. IMPORTACIÓN DE LIBRERÍAS, SAMPLEO DE DATOS, ANALISIS EXPLORATORIO

In [ ]:
# import json
# with open("kaggle.json","w") as filepath:
#   json.dump({'key': 'b0456b6bdf73e205861cdca4365786b3', 'username': 'joaquin96'}, filepath)
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets download joaquin96/properati-digital-house-2021 -o -q --unzip
# !pip install unidecode -q

In [ ]:
#Importamos las librerías que vamos a usar para limpiar y recorrer el data set.

import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import unidecode #from unidecode import unidecode
from scipy.stats import iqr
pd.set_option('display.float_format', lambda x: '%.2f' % x) #convertir floats con notacion cientifica a dos decimales

In [ ]:
#Leemos el data set y sampleamos 5 registros para explorar datos y chequear que se haya importado ok

data = pd.read_csv('properatti.csv', sep = ',')
print(data.shape)
data.sample(5)

In [ ]:
# vemos columnas que tenemos y tipos de datos

data.columns

In [ ]:
# vemos que la columna unnamed: 0 coincide con el indice, por lo que no la vamos a cosniderar en el dataset final

#data.drop('Unnamed: 0', axis = 1, inplace = True)
#data.sample(3)

In [ ]:
#hay registros duplicados? 

#No se ven duplicados en el general

data.duplicated().sum()

In [ ]:
#exploremos algunas columnas y sus valores 
data['operation'].value_counts()

#vemos que esta columna no nos aporta valor porque son todos iguales, por lo tanto, no la vamos a considerar en el data final. 

In [ ]:
data['property_type'].value_counts()

In [ ]:
data['country_name'].value_counts()

In [ ]:
data['state_name'].value_counts()

In [ ]:
#chequeamos el porcentaje de nulos de cada campo

100 * (data.isnull().sum() / data.shape[0])

<a id="section_ubicacion"></a>
### AJUSTE DE COLUMNAS REFERIDAS A LA UBICACIÓN. 
SPLIT DE "place_with_parent_names"


In [ ]:
#Acomodar los datos de place_with_parent_names

#Primero pasamos todos los valores a una lista aplicando split, y convertimos ese objeto en DataFrame

place_with = data['place_with_parent_names'].apply(lambda x: x if x is np.NaN else x.split('|'))
place_with_df = pd.DataFrame(place_with)

#Aplicamos pd.Serie a cada valor de place_with

tags = place_with_df['place_with_parent_names'].apply(pd.Series)

#Renombramos las columnas 

place_with_parent_name_df = tags.rename(columns = lambda x : 'Ubicacion_' + str(x))


In [ ]:
#Ahora que tenemos un DF con place_with_parent_names en columnas, trabajamos sobre este para que quede limpio y después concatenarlo al data ser original 
place_with_parent_name_df.sample(3)

In [ ]:
#La columna Ubicacion_0 y Ubicación_ 6 no tienen información

place_with_parent_name_df['Ubicacion_0'].value_counts()
place_with_parent_name_df['Ubicacion_6'].value_counts()

#Vemos que "ubicación 1" es Argetina para todos los registros al igual que la columna "Country_Name" de Data. 
data['country_name'].value_counts() == place_with_parent_name_df['Ubicacion_1'].value_counts()

#Vemos que "ubicación 2" es para todos los registros igual que la columna "state_name" de Data. 
place_with_parent_name_df['Ubicacion_2'].value_counts() == data['state_name'].value_counts()

#Vemos que Ubicación_3, Ubicación_4 y Ubicación_5 si nos traen información útil.
place_with_parent_name_df['Ubicacion_3'].value_counts()
place_with_parent_name_df['Ubicacion_4'].value_counts()
place_with_parent_name_df['Ubicacion_5'].value_counts()

#Vamos a quedarnos sólo con estas 3 series en un DF que luego de imputar valores a los nulos, vamos a concatenar a Data.
place_with_parent_name_final = place_with_parent_name_df[['Ubicacion_3', 'Ubicacion_4', 'Ubicacion_5']].copy()

#Renombramos como Barrio_1 Barrio_2 y Barrio_3
dict_ubi = {'Ubicacion_3': 'Barrio_1', 'Ubicacion_4': 'Barrio_2', 'Ubicacion_5': 'Barrio_3' }
place_with_parent_name_final.rename(mapper = dict_ubi, axis = 1, inplace = True)
place_with_parent_name_final.sample(3)

In [ ]:
#veamos ahora los nulos de place_with_parent_name_final para imputar antes de unirlo a data

#En Barrio 1, Barrio_2 y Barrio_3 tenemos campos vacios pero que no aparecen como nulos, por eso vamos a reemplazar y convertirlos en np.NaN para despues poder impurtar con FillNa
place_with_parent_name_final['Barrio_1'].replace("", np.NaN, inplace = True)
place_with_parent_name_final['Barrio_2'].replace("", np.NaN, inplace = True)
place_with_parent_name_final['Barrio_3'].replace("", np.NaN, inplace = True)


#ahora si, a lo que no es nulo le vamos a poner "Sin definir"
place_with_parent_name_final.fillna("Sin definir", inplace = True)

In [ ]:
#joineamos data y place_with_parent_name_final 
data = pd.concat([data, place_with_parent_name_final], axis = 1)
print(data.shape)
data.sample(3)

In [ ]:
#en data nos quedaron columnas que ya no nos son útiles, porque las tenemos en Barrio_1, Barrio_2 y Barrio_3. Estas son place_name y place_with_parent_names 


In [ ]:
#vemos que en state_name tenemos Bs.As para unos y Buenos Aires para otros. Lo mejor sería unificar todo a una sola nomenclatura 
#lo hacemos con el metodo replace 

data['state_name'].replace('Buenos Aires Costa Atlántica', 'Bs.As. Costa Atlántica', inplace = True)
data['state_name'].replace('Buenos Aires Interior', 'Bs.As. Interior', inplace = True)
data['state_name'].value_counts()


<a id="section_descripcion"></a>
### CAMPO DESCRIPTION - LIMPIEZA DE NULOS, CASEFOLD, UNICODE

In [ ]:
#En primer lugar vamos a convertir todos los caracteres de "Descripción" a minúscula, para facilitar el matecheo con regex. 

# sacamos los nulos y ponemos un ""
data.loc[data.description.isna(),"description"] = ""

# pasamos todo a minusculas y a unicode
data["description_Clean"]= data['description'].apply(lambda x: unidecode.unidecode(x.casefold()))
data["description_Clean"]= data['description_Clean'].replace('\s+', ' ', regex=True)
data["description_Clean"].sample()

<a id="section_rooms"></a>
### LIMPIEZA DEL CAMPO 'rooms', IMPUTACIÓN DE DATOS FALTANTES.

In [ ]:
data['rooms'].isnull().sum()

In [ ]:
#Vamos a usar regex para buscar información en el campo 'description' y lo que encontremos lo vamos a poner en 'rooms_Clean'

#Regex en descrpition para buscar más de un ambiente. 

description_serie = data['description_Clean']
rooms_pattern = "(?P<ambientes>\d)\s(amb)"
rooms_pattern_regex = re.compile(rooms_pattern)

rooms_in_description = description_serie.apply(lambda x: x if x is np.NaN  else rooms_pattern_regex.search(x))
rooms_in_description_mask_not_null = rooms_in_description.notnull()
data.loc[rooms_in_description_mask_not_null, 'rooms_Clean'] = rooms_in_description[rooms_in_description_mask_not_null].apply(lambda x: x.group('ambientes'))

print('Primera imputación. Nulos restantes', data['rooms_Clean'].isnull().sum())

#Regex en descrpition para buscar la palabra 'monoambiente'. 

monoambiente_pattern = r'monoambiente'
monoambiente_pattern_regex = re.compile(monoambiente_pattern)

monoambiente_in_description = description_serie.apply(lambda x: x if x is np.NaN  else monoambiente_pattern_regex.search(x))
monoambiente_in_description_mask_not_null = monoambiente_in_description.notnull()
data.loc[monoambiente_in_description_mask_not_null, 'rooms_Clean'] = 1

print('Segunda imputación. Nulos restantes', data['rooms_Clean'].isnull().sum())

#Ahora vamos a tomar los valores no nulos de 'rooms' y ponerlos en 'rooms_Clean'
data.loc[(data['rooms'].notnull()), 'rooms_Clean'] = data['rooms']

#pasamos rooms_Clean a float, y al poner errors = 'ignore' hay algunos valores que se pueden convertir en 0. Por lo que después de esto vamos a convertirlos en np.NaN

data['rooms_Clean'] = data['rooms_Clean'].astype(float, errors = 'ignore')


print('Tercera imputación. Nulos restantes', data['rooms_Clean'].isnull().sum())



In [ ]:
#Veamos un poco la disperción de rooms_Clean antes de imputar valores por mediana

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "rooms_Clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "rooms_Clean");
    
data.groupby('property_type')['rooms_Clean'].describe()


In [ ]:
#mas allá de haber conseguido imputar muchos valores con las regex de Rooms, todavía tenemos mucho null. 
#Vamos a hacer una imputación que sea por la mediana de ambientes según "property_type" y "state_name"
#agrupado por property_type

#Creamos una serie que tenga la mediana por 'property_type' y 'state_name'
grouped_by_rooms_clean = data.groupby(['property_type','state_name' ])['rooms_Clean'].transform('median').round(0)

#usamos esa serie para completar los nulos en 'rooms_clean'
data.loc[(data['rooms_Clean'].isnull()), 'rooms_Clean'] = grouped_by_rooms_clean

print('Cuarta imputación. Nulos restantes', data['rooms_Clean'].isnull().sum())

#vemos que con esta imputación los nulos bajaron a 112. Esto se debe a que algunos 'state_name' no tienen todos los property type. 

#Vamos a completar estos 112 faltantes imputando por la mediana pero esta vez solo considerando el 'property_type'

grouped_by_rooms_clean_2 =  data.groupby(['property_type'])['rooms_Clean'].transform('median').round(0)

data.loc[(data['rooms_Clean'].isnull()), 'rooms_Clean'] = grouped_by_rooms_clean_2

print('Quinta imputación. Nulos restantes', data['rooms_Clean'].isnull().sum())

#vemos que con esta imputación los nulos bajaron a 38. Vamos a completar estos faltantes imputando por la mediana ya sin agrupar

data.loc[(data['rooms_Clean'].isnull()), 'rooms_Clean'] = data['rooms_Clean'].median(skipna = True).round(0)

print('Sexta imputación. Nulos restantes', data['rooms_Clean'].isnull().sum())


In [ ]:
#vemos la distribución post imputar por la mediana

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "rooms_Clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "rooms_Clean");
    
data.groupby('property_type')['rooms_Clean'].describe()


<a id="section_surface"></a>
### LIMPIEZA DE CAMPO 'surface_total_in_m2' y 'surface_covered_in_m2'

In [ ]:
#regex para extraer m2 totales y cubiertos

metros_extract = data['description_Clean'].str.extract("([\d.,?]+)(\s+)?(m2|metros|mts|ms)", re.IGNORECASE)
data["surface_total_in_m2_Clean"] = np.where(data["surface_total_in_m2"].isna(), metros_extract[0], data["surface_total_in_m2"])

metros_cub_extract = data['description_Clean'].str.extract("([\d.,?]+)(\s+)?(m2|metros|mts|ms)(\s+)?(cubiertos)", re.IGNORECASE)
data["surface_covered_in_m2_Clean"] = np.where(data["surface_covered_in_m2"].isna(), metros_cub_extract[0], data["surface_covered_in_m2"])


In [ ]:
#convertimos nulos de las columnas de m2 totales y cubiertos en 0
data["surface_covered_in_m2_Clean"]= data["surface_covered_in_m2_Clean"].apply(lambda x: x if x is not np.NaN else 0)
data["surface_total_in_m2_Clean"]= data["surface_total_in_m2_Clean"].apply(lambda x: x if x is not np.NaN else 0)
data["surface_covered_in_m2_Clean"]=data["surface_covered_in_m2_Clean"].apply(lambda x: str(x).replace(",","."))
data["surface_total_in_m2_Clean"]=data["surface_total_in_m2_Clean"].apply(lambda x: str(x).replace(",","."))

#convertimos str a int, los errores se convierten en nulos, y estos nuevamente a 0
data["surface_total_in_m2_Clean"]=pd.to_numeric(data["surface_total_in_m2_Clean"], errors="coerce", downcast="integer")
data["surface_covered_in_m2_Clean"]=pd.to_numeric(data["surface_covered_in_m2_Clean"], errors="coerce", downcast="integer")
data["surface_covered_in_m2_Clean"]= data["surface_covered_in_m2_Clean"].apply(lambda x: x if x is not np.NaN else 0)
data["surface_total_in_m2_Clean"]= data["surface_total_in_m2_Clean"].apply(lambda x: x if x is not np.NaN else 0)

In [ ]:
#verificamos que la superficie cubierta no supere la superficie total

surface_check = data.loc[:,["surface_covered_in_m2_Clean","surface_total_in_m2_Clean"]]

#donde el total es menor al area cubierta, utilizamos area cubierta, si no usamos total.

surface_check["surface_total_in_m2_Clean"]= np.where(surface_check.surface_total_in_m2_Clean < surface_check.surface_covered_in_m2_Clean, surface_check.surface_covered_in_m2_Clean, surface_check.surface_total_in_m2_Clean)

#verificamos que np.where sea aplicado correctamente 
surface_check["dif"]=(surface_check["surface_total_in_m2_Clean"]-surface_check["surface_covered_in_m2_Clean"])
print("nº de observaciones donde cubierta>total:",surface_check[surface_check.dif<0].shape[0])

#aplicamos el codigo al dataframe original
data["surface_total_in_m2_Clean"]= np.where(data.surface_total_in_m2_Clean < data.surface_covered_in_m2_Clean, data.surface_covered_in_m2_Clean, data.surface_total_in_m2_Clean)



In [ ]:
print("observaciones totales:",data.shape[0],"\n")
print("superficie_Clean total:", data.loc[data.surface_total_in_m2_Clean>0, "surface_total_in_m2_Clean"].shape[0])
print("superficie total:", data.loc[data.surface_total_in_m2>0, "surface_total_in_m2"].shape[0])
print("")
print("superficie_cubierta_Clean total:", data.loc[data.surface_covered_in_m2_Clean > 0, "surface_covered_in_m2_Clean"].shape[0])
print("superficie_cubierta total:", data.loc[data.surface_covered_in_m2 > 0, "surface_covered_in_m2"].shape[0])
print('Nulos en surface_covered_in_m2_Clean', (data['surface_covered_in_m2_Clean'] == 0).sum())
print('Nulos en surface_total_in_m2_Clean', (data['surface_total_in_m2_Clean'] == 0).sum())

In [ ]:
data[["surface_covered_in_m2_Clean","surface_total_in_m2_Clean"]].describe()

In [ ]:
#veamos como quedan los valores antes de imputar por la mediana

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "surface_total_in_m2_Clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "surface_total_in_m2_Clean");
    
data.groupby('property_type')[('surface_total_in_m2_Clean')].describe()


In [ ]:
#Vamos a imputar por la mediana, agrupanto por property_type, state_name y rooms_clean

#Creamos una serie que tenga la mediana por 'property_type', 'state_name' y 'rooms_clean'
grouped_by_m2total_clean = data.groupby(['property_type','state_name','rooms_Clean'])['surface_total_in_m2_Clean'].transform('median')
grouped_by_m2covered_clean = data.groupby(['property_type','state_name',"rooms_Clean"])['surface_covered_in_m2_Clean'].transform('median')

#usamos esas series para completar los nulos en 'surface_total_in_m2_Clean' y 'surface_covered_in_m2_Clean'
data.loc[(data['surface_total_in_m2_Clean'] == 0), 'surface_total_in_m2_Clean'] = grouped_by_m2total_clean
data.loc[(data['surface_covered_in_m2_Clean'] == 0), 'surface_covered_in_m2_Clean'] = grouped_by_m2covered_clean

print('Nulos en surface_covered_in_m2_Clean', (data['surface_covered_in_m2_Clean'] == 0).sum())
print('Nulos en surface_total_in_m2_Clean', (data['surface_total_in_m2_Clean'] == 0).sum())

In [ ]:
#segunda limpieza con property type y rooms_clean

grouped_by_m2total_clean2 = data.groupby(['property_type','rooms_Clean'])['surface_total_in_m2_Clean'].transform('median')
grouped_by_m2covered_clean2 = data.groupby(['property_type','rooms_Clean'])['surface_covered_in_m2_Clean'].transform('median')

data.loc[(data['surface_total_in_m2_Clean'] == 0), 'surface_total_in_m2_Clean'] = grouped_by_m2total_clean2
data.loc[(data['surface_covered_in_m2_Clean'] == 0), 'surface_covered_in_m2_Clean'] = grouped_by_m2covered_clean2

print('Nulos en surface_covered_in_m2_Clean', (data['surface_covered_in_m2_Clean'] == 0).sum())
print('Nulos en surface_total_in_m2_Clean', (data['surface_total_in_m2_Clean'] == 0).sum())


In [ ]:
#segunda limpieza solo con property_type
grouped_by_m2total_clean3 = data.groupby(['property_type'])['surface_total_in_m2_Clean'].transform('median')
grouped_by_m2covered_clean3 = data.groupby(['property_type'])['surface_covered_in_m2_Clean'].transform('median')

data.loc[(data['surface_total_in_m2_Clean']==0)|(data['surface_total_in_m2_Clean'].isna()), 'surface_total_in_m2_Clean'] = grouped_by_m2total_clean3
data.loc[(data['surface_covered_in_m2_Clean']==0|(data['surface_covered_in_m2_Clean'].isna())), 'surface_covered_in_m2_Clean'] = grouped_by_m2covered_clean3

print('Nulos en surface_covered_in_m2_Clean', (data['surface_covered_in_m2_Clean'] == 0).sum())
print('Nulos en surface_total_in_m2_Clean', (data['surface_total_in_m2_Clean'] == 0).sum())

In [ ]:
#teniamos 2616 observaciones con menos de 30m2
data.surface_total_in_m2_Clean[data.surface_total_in_m2_Clean<30].count()

In [ ]:
#tenemos observaciones con menos de 30m2, por lo tanto tenemos que eliminarlas.

#Creamos una serie que tenga la mediana por 'property_type' y 'state_name'
grouped_by_surface_total= data.groupby(['property_type'])['surface_total_in_m2_Clean'].transform('median')

#usamos esas series para completar los nulos en 'price_clean'
data.loc[(data['surface_total_in_m2_Clean'] < 30), 'surface_total_in_m2_Clean'] = grouped_by_surface_total

print('Datos menores a 30:', (data["surface_total_in_m2_Clean"] < 30).sum())


In [ ]:
data['surface_total_in_m2_Clean'].isna().sum()

In [ ]:
data["surface_covered_in_m2_Clean"].isna().sum()

In [ ]:
#Creamos una serie que tenga la mediana por 'property_type' y 'state_name'
grouped_by_surface_covered= data.groupby(['property_type'])['surface_covered_in_m2_Clean'].transform('median')

#usamos esas series para completar los nulos en 'price_clean'
data.loc[(data['surface_covered_in_m2_Clean'] < 30), 'surface_covered_in_m2_Clean'] = grouped_by_surface_covered

print('Datos menores a 30:', (data["surface_covered_in_m2_Clean"] < 30).sum())


In [ ]:
#Vemos como quedan los datos despues de imputar por la mediana

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "surface_total_in_m2_Clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "surface_total_in_m2_Clean");
    
data.groupby('property_type')[('surface_total_in_m2_Clean')].describe()



<a id="section_price"></a>
### LIMPIEZA DE CAMPO 'price', 'price_aprox_usd' y 'currency'

In [ ]:
# comparo si todos los precios que estan en USD coinciden con los precios de la columna price_aprox_usd
mismos_precios = data.loc[data['currency']=='USD','price'] == data.loc[data['currency']=='USD','price_aprox_usd']
all(mismos_precios) 


In [ ]:
#Verificamos el tipo de cambio dividiendo la columna de price en ARS por la columna price_aprox_usd
Lista_tipo_cambio = data.loc[data['currency']=='ARS','price_aprox_usd'] / data.loc[data['currency']=='ARS','price']
# Graficamos los tipos y vemos que no hay gran variabilidad
plt.plot(Lista_tipo_cambio)
plt.show()
# Vemos su descripción estadistica
Lista_tipo_cambio.describe()
print('Media de tipo de cambio USD a ARS', Lista_tipo_cambio.mean())

In [ ]:
# Vamos a mantener el precio si este está en USD y si está en ARS lo vamos a pasar multiplicado por el tipo de cambio
data["price_clean"] = np.where(data["currency"]=="ARS",data["price"]*(Lista_tipo_cambio.mean()),data["price"])
data.loc[data.price_clean.isna(), "price_clean"] = 0

In [ ]:
print(data['currency'].value_counts())

#droperamos las filas que en currency no son ni USD ni ARS (son 3 filas que tienen PEN y UYU)
data = data[(data.currency=="USD") | (data.currency=="ARS") | (data.currency.isnull())].copy()

#Currency ya no nos aporta mas info, así que deja de ser parte del dataset final, porque vamos a tener todo en usd. 
print('-------------')
print(data['currency'].value_counts())

Regex para sacar precios en USD de la descripcion y el titulo

In [ ]:
# Tenemos que tener en cuenta que al hacer un Regex con los precios podemos encontrar varias cosas como ser el precio por m2, lo que vamos a eliminar como outliers luego
title_serie = data['title']
usd_pattern = "(U|u)\$.\s\d*.?(\d*)"
priceusd_fromtitle_pattern_regex = re.compile(usd_pattern)

priceusd_in_title = title_serie.apply(lambda x: x if x is np.NaN else priceusd_fromtitle_pattern_regex.search(x))

priceusd_in_title_mask_not_null = priceusd_in_title.notnull()
data.loc[priceusd_in_title_mask_not_null, 'priceusd_fromtitle_notclean'] = priceusd_in_title[priceusd_in_title_mask_not_null].apply(lambda x: x.group())
data.loc[priceusd_in_title_mask_not_null, ["title", "priceusd_fromtitle_notclean"]].head(5)

In [ ]:
# Transformo la columna para poder usar lstrip
data['priceusd_fromtitle_notclean'] = data['priceusd_fromtitle_notclean'].astype(str)
data['priceusd_fromtitle_notclean'] = data['priceusd_fromtitle_notclean'].map(lambda x: x.lstrip('Uu$SsDd'))
data['priceusd_fromtitle_notclean'] = data['priceusd_fromtitle_notclean'].map(lambda x: x.replace('.', ''))

# Muestro el resultado
data.loc[priceusd_in_title_mask_not_null, ["title", "priceusd_fromtitle_notclean"]].head(5)

In [ ]:
# Pasamos la columna de Object a float
data["priceusd_fromtitle_notclean"] = data["priceusd_fromtitle_notclean"].apply(lambda x: x if x is not np.NaN else 0)
data["priceusd_fromtitle_notclean"] = pd.to_numeric(data["priceusd_fromtitle_notclean"], errors="coerce", downcast="integer")
data["priceusd_fromtitle_notclean"] = data["priceusd_fromtitle_notclean"].apply(lambda x: x if x is not np.NaN else 0)
data["priceusd_fromtitle_notclean"].describe()


In [ ]:
# Tenemos que tener en cuenta que al hacer un Regex con los precios podemos encontrar varias cosas como ser el precio por m2, lo que vamos a eliminar como outliers luego
description_serie = data['description']
usd_pattern = "(U|u)\$.\s\d*.?(\d*)"
priceusd_fromdescription_pattern_regex = re.compile(usd_pattern)

priceusd_in_description = description_serie.apply(lambda x: x if x is np.NaN else priceusd_fromdescription_pattern_regex.search(x))

priceusd_in_description_mask_not_null = priceusd_in_description.notnull()
data.loc[priceusd_in_description_mask_not_null, 'priceusd_fromdescription_notclean'] = priceusd_in_description[priceusd_in_description_mask_not_null].apply(lambda x: x.group())
data.loc[priceusd_in_description_mask_not_null, ["title", "priceusd_fromdescription_notclean"]].head(5)

In [ ]:
# Transformo la columna para poder usar lstrip
data['priceusd_fromdescription_notclean'] = data['priceusd_fromdescription_notclean'].astype(str)
data['priceusd_fromdescription_notclean'] = data['priceusd_fromdescription_notclean'].map(lambda x: x.lstrip('Uu$SsDd'))
data['priceusd_fromdescription_notclean'] = data['priceusd_fromdescription_notclean'].map(lambda x: x.replace('.', ''))

# Muestro el resultado
data.loc[priceusd_in_description_mask_not_null, ["title", "priceusd_fromdescription_notclean"]].head(5)

In [ ]:
# Pasamos la columna priceusd_fromdescription_notclean de Object a float
data["priceusd_fromdescription_notclean"] = data["priceusd_fromdescription_notclean"].apply(lambda x: x if x is not np.NaN else 0)
data["priceusd_fromdescription_notclean"] = pd.to_numeric(data["priceusd_fromdescription_notclean"], errors="coerce", downcast="integer")
data["priceusd_fromdescription_notclean"] = data["priceusd_fromdescription_notclean"].apply(lambda x: x if x is not np.NaN else 0)
data["priceusd_fromdescription_notclean"].describe()
# Muestro el resultado
data.loc[priceusd_in_description_mask_not_null, ["title", "priceusd_fromdescription_notclean"]].head(5)

In [ ]:
# Uno las 2 columnas de priceusd_fromdescription_notclean y priceusd_fromtitle_notclean

# Inicio una columna toda en cero
data['priceusd_fromdescription_and_fromtitle'] = 0
# Si detecto un valor en priceusd_fromtitle_notclean > 1000 se lo asigno a priceusd_fromdescription_and_fromtitle sino dejo cero
data["priceusd_fromdescription_and_fromtitle'"] = np.where(data["priceusd_fromtitle_notclean"]>1000,data["priceusd_fromtitle_notclean"],0)
# Si detecto un valor en priceusd_fromdescription_notclean se lo asigno a asigno a priceusd_fromdescription_and_fromtitle sino dejo el que estaba
data["priceusd_fromdescription_and_fromtitle'"] = np.where(data["priceusd_fromdescription_notclean"]>1000,data["priceusd_fromdescription_notclean"],data["priceusd_fromdescription_and_fromtitle'"])

In [ ]:
data['price_clean']=np.where(data["price_clean"]==0,data['priceusd_fromdescription_and_fromtitle'],data["price_clean"])

In [ ]:
#Vemos como quedan los datos antes de imputar por la mediana

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "price_clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "price_clean")
    
data.groupby('property_type')[('price_clean')].describe()


In [ ]:
#imputamos price_clean por la mediana

print('Nulos en price_clean', (data["price_clean"] == 0).sum())

#Creamos una serie que tenga la mediana por 'property_type','rooms_clean' y 'state_name'
grouped_by_price_clean = data.groupby(['property_type','state_name','rooms_Clean'])['price_clean'].transform('median')

#usamos esas series para completar los nulos en 'price_clean'
data.loc[(data['price_clean'] == 0), 'price_clean'] = grouped_by_price_clean

print('Nulos en price_clean', (data["price_clean"] == 0).sum())


In [ ]:
#Creamos una serie que tenga la mediana por 'property_type' y 'state_name'
grouped_by_price_clean2 = data.groupby(['property_type','state_name'])['price_clean'].transform('median')

#usamos esas series para completar los nulos en 'price_clean'
data.loc[(data['price_clean'] == 0), 'price_clean'] = grouped_by_price_clean2

print('Nulos en price_clean', (data["price_clean"] == 0).sum())


In [ ]:
#Creamos una serie que tenga la mediana por 'property_type' 
grouped_by_price_clean3 = data.groupby(['property_type'])['price_clean'].transform('median')

#usamos esas series para completar los nulos en 'price_clean'
data.loc[(data['price_clean'] == 0), 'price_clean'] = grouped_by_price_clean3

print('Nulos en price_clean', (data["price_clean"] == 0).sum())

In [ ]:
#price_usd_per_m2_clean

#utilizamos los valores que tenemos, si no los sacamos por formula
data["price_usd_per_m2_Clean"]=np.where(data.price_usd_per_m2>0, data.price_usd_per_m2, data["price_clean"]/data["surface_total_in_m2_Clean"])

#convertimos np.NaN e inf en 0
data["price_usd_per_m2_Clean"]=data["price_usd_per_m2_Clean"].replace(np.NaN,0).replace(np.inf,0)

data["price_usd_per_m2_Clean"].astype(np.float).describe()

In [ ]:
#Vemos como quedan los datos de price_clean despues de imputar por la mediana

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "price_clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "price_clean");
    
data.groupby('property_type')[('price_clean')].describe()


In [ ]:
#Vemos como quedan los datos price_usd_per_m2_Clean 

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "price_usd_per_m2_Clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "price_usd_per_m2_Clean");
    
data.groupby('property_type')[('price_usd_per_m2_Clean')].describe()


<a id="section_drop"></a>
### DROP DE OUTLIERS Y NULOS

In [ ]:
#definimos una función para dropear los outliers definida por el IQR
#Ahora la uso solo en rooms pero la idea es usarla en todo. 
registros_totales = data.shape[0]
col_clean = ['rooms_Clean', 'surface_total_in_m2_Clean','surface_covered_in_m2_Clean', 'price_clean','price_usd_per_m2_Clean']

def iqr_column(columna): #input: data.columna 
    iqr_temp = iqr(columna[columna>1])*1.5               #iqr
    Q3_temp = columna[columna>1].quantile(0.75)+iqr_temp #upper limit
    Q1_temp = columna[columna>1].quantile(0.25)-iqr_temp #lower limit
    print(columna.name)
    print("iqr:",iqr_temp)
    print("upper:",Q3_temp)
    print("lower:",Q1_temp)
    return Q3_temp,Q1_temp


for col in col_clean:
    col = data[col]
    registros_campo=col.shape[0]
    upper_temp, lower_temp = iqr_column(col)
    data.drop(data[(col>upper_temp)|(col<lower_temp)].index,axis=0,inplace=True)
    print('Se droperon', (registros_campo- data.shape[0]),  'outliers del campo', col.name)
    print("")

print('total dropeado:', registros_totales - data.shape[0])


<a id="section_graficos"></a>
### GRÁFICOS POST DROPEO FINAL

In [ ]:
data[data[col_clean]>0].loc[:,col_clean].describe()

In [ ]:
#Vemos como queda la disperción después de dropear outliers en rooms

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "rooms_Clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "rooms_Clean");
    
data.groupby('property_type')['rooms_Clean'].describe()


In [ ]:
#Vemos como queda la disperción después de dropear outliers en rooms

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "rooms_Clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "rooms_Clean");
    
data.groupby('property_type')['rooms_Clean'].describe()


In [ ]:
#Vemos como queda la disperción después de dropear outliers en surface_total_in_m2_Clean

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "surface_total_in_m2_Clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "surface_total_in_m2_Clean");
    
data.groupby('property_type')[('surface_total_in_m2_Clean')].describe()


In [ ]:
#Vemos como queda la disperción después de dropear outliers en price_clean

with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "price_clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "price_clean");
    
data.groupby('property_type')[('price_clean')].describe()


In [ ]:
#Vemos como queda la disperción después de dropear outliers en price_usd_per_m2_Clean


with sns.axes_style(style='ticks'):
    g = sns.catplot(x = 'property_type', y = "price_usd_per_m2_Clean", data=data, kind="boxen")
    g.set_axis_labels("Property_Type", "price_usd_per_m2_Clean");
    
data.groupby('property_type')[('price_usd_per_m2_Clean')].describe()


In [ ]:
data.columns #'rooms_Clean', 'surface_total_in_m2_Clean','surface_covered_in_m2_Clean', 'price_clean',price_usd_per_m2_Clean

In [ ]:
data.isna().sum()

In [ ]:
data.dropna(subset=['property_type','Barrio_1', 'Barrio_2', 'Barrio_3',
       'description_Clean', 'rooms_Clean', 'surface_total_in_m2_Clean',
       'surface_covered_in_m2_Clean', 'price_clean', 'price_usd_per_m2_Clean'],inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
data_final=data[['property_type','state_name','Barrio_1', 'Barrio_2', 'Barrio_3',
       'description_Clean', 'rooms_Clean', 'surface_total_in_m2_Clean',
       'surface_covered_in_m2_Clean', 'price_clean', 'price_usd_per_m2_Clean']]

print(data_final.isna().sum())
data_final


In [ ]:
#Veamos la correlación entre las variables que elegimos dejar en data final

sns.heatmap(data_final.corr(), annot=True, vmin=-1, cmap='Blues');

In [ ]:
sns.pairplot(data_final);